In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
r = requests.get('https://www.akademisains.gov.my/asmsj/published-articles/')

In [6]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = [a_.get('href') for a_ in a if 'asmsj/asm' in a_.get('href')]
pages = sorted(list(set(pages)))
len(pages)

48

In [7]:
pages

['https://www.akademisains.gov.my/asmsj/asm-sc-j-11-special-issue-2-2018-for-sanrem-natural-resources/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-11-special-issue-3-2018-for-sanrem-environmental-management/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-12-2019/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-12-special-issue-1-2019-for-iqrac2018/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-12-special-issue-2-2019-malaysia-in-space/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-special-issue-20171-ict-bio/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-vol-12-special-issue-3-2019-for-icst2018/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-vol-12-special-issue-4-2019-for-icse2018/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-vol-12-special-issue-5-2019-for-icoaims2019/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-vol-12-special-issue-6-2019-for-sksm26/',
 'https://www.akademisains.gov.my/asmsj/asm-sc-j-vol-13-2020/',
 'https://www.akademis

In [10]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if 'article/' in a_.get('href', '')]
    urls.extend(a)
    
len(urls)

100%|███████████████████████████████████████████| 48/48 [02:35<00:00,  3.23s/it]


2875

In [12]:
urls2 = sorted(list(set(urls)))
len(urls2)

811

In [15]:
pdfs = []
for p in tqdm(urls2):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if '/?mdocs-file=' in a_.get('href', '')]
    pdfs.extend(a)

100%|█████████████████████████████████████████| 811/811 [44:41<00:00,  3.31s/it]


In [17]:
pdfs = sorted(list(set(pdfs)))
len(pdfs)

811

In [18]:
!mkdir akademisains.gov.my

In [19]:
import os

for i in tqdm(range(len(pdfs))):
    filename = os.path.join('akademisains.gov.my', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 811/811 [33:24<00:00,  2.47s/it]


In [20]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [21]:
files = glob('akademisains.gov.my/*.pdf')

with open('akademisains.gov.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|█████████████████████████████████████████| 811/811 [00:52<00:00, 15.57it/s]
